In [8]:
experiment_name = "vision"
results_dir = '/om2/user/xboix/robustness/convex_robustness/'
%cd /om2/vast/cbmm/xboix/src/convex_adversarial

/om2/vast/cbmm/xboix/src/convex_adversarial


In [9]:
import seaborn as sns
sns.set_theme(style="ticks")
sns.set_context("poster")

import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import os 

import numpy as np
import pandas
    
import runs.config_experiments_vision as run
experiment_list = run.config_experiments(results_dir, create_json=False)

28 config files created


In [10]:
name_attacks = ["l1_pgd_norm", "l1_fgm_norm", "linf_pgd", "l2_pgd_norm", "linf_fgsm", "l2_fgm_norm"]

In [11]:
list_entries = []
for net in ['ThreeLayer']:
    for dataset_id,dataset_name in zip([0],['MNIST']):

        #Find an experiment of the dataset_id to get the epsilons
        for idx,_ in enumerate(experiment_list):
            if experiment_list[idx]['data_set_id'] == dataset_id: 
                break
        
        for attack in name_attacks:

            file_name = results_dir + experiment_list[idx]['model_name'] + '/results/acc_' + 'val' + '_' + attack + '.pkl'
            if not os.path.isfile(file_name):
                print("Missing!! " + file_name)
                continue
            with open(file_name, 'rb') as f:
                tmp = pickle.load(f)
                

            for idx_epsilon, cv_epsilon in enumerate(list(tmp.keys())):
                # Hash table of parameters
                parameters = {"epsilon": {}, "standarize": {}, "backbone": {}, "initial_learning_rate": {}, 
                              "robust_training": {}, "type_robust": {}, "epsilon_pgd_training":{}}

                to_exclude = [0]
                experiment_list_tmp = [element for i, element in enumerate(experiment_list) if i not in to_exclude]
                for exp in experiment_list_tmp:
                    if not exp['data_set_id'] == dataset_id: 
                        continue
                    for kk in parameters.keys():
                        if exp[kk] in parameters[kk]:
                            parameters[kk][exp[kk]].append(int(exp["model_name"]))
                        else:
                            parameters[kk][exp[kk]] = [int(exp["model_name"])]


                # For all methods, do cross-val and create an entry of the results
                backbones = [net]

                for backbone in backbones:
                    for robust_training in [False]:
                        if robust_training:
                            type_robust_trainings = ['convex']
                        else:
                            type_robust_trainings = ['none']
                            
                        for type_robust in type_robust_trainings:

                            if robust_training==False:
                                ids = list(set(parameters["backbone"][backbone]) & 
                                           set(parameters["robust_training"][False]))
                            else:
                                ids = list(set(parameters["backbone"][backbone]) & 
                                       set(parameters["robust_training"][True]))
                            
                            #print(ids)
                            if ids == []:
                                continue
                            

                            # Cross-validation among learning rates and epsilons:
                            best_acc = -1
                            best_id = ids[0]
                            for id in ids:
                                if ((robust_training == False) & (backbone==net) \
                                        & (experiment_list[id]['training_batch_size']==256)):
                                    continue
                                    
                                file_name = results_dir + experiment_list[id]['model_name'] + '/results/acc_' + 'val' + '_' + attack + '.pkl'
                                if not os.path.isfile(file_name):
                                    print("Missing!! " + file_name)
                                    continue

                                with open(file_name, 'rb') as f:
                                    tmp_acc = pickle.load(f)

                                acc = list(tmp_acc.values())[idx_epsilon] #tmp_acc[cv_epsilon]
                                if acc>best_acc:
                                    best_id = id
                                    best_acc = acc

                            if best_acc == -1:
                                continue

                            if (robust_training == False) & (backbone==net):  
                                name_legend = 'vanilla'
                            else:
                                if type_robust=='convex':
                                    name_legend = 'Convex'

                            entry = {"learning_rate": experiment_list[best_id]['initial_learning_rate'],
                                     "net": net,
                                     "dataset": dataset_name,
                                     "standarize": experiment_list[best_id]['standarize'],
                                    "robust_training": name_legend,
                                    "epsilon": experiment_list[best_id]['epsilon'],
                                    "epsilon_pgd_training": experiment_list[best_id]['epsilon_pgd_training']}

                            dataset = "test"
                            entry["attack"] = attack
                            entry["experiment_id"] = best_id

                            with open(results_dir + experiment_list[best_id]['model_name'] + '/results/acc_' + dataset + '_' + 
                                attack + '.pkl', 'rb') as f:
                                tmp_acc = pickle.load(f)

                            entry["accuracy"] =  100*tmp_acc[cv_epsilon]#(list(tmp_acc.values())[idx_epsilon]) #tmp_acc[cv_epsilon]
                            entry["test_epsilon"] = cv_epsilon

                            list_entries.append(entry.copy())

df_results = pd.DataFrame.from_dict(list_entries) 




In [12]:
df_results[df_results.attack=='linf_pgd']

,learning_rate,net,dataset,standarize,robust_training,epsilon,epsilon_pgd_training,attack,experiment_id,accuracy,test_epsilon
40,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,98.1250,0.0000
41,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,98.1250,0.0002
42,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,98.1250,0.0005
43,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,98.1250,0.0008
44,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,98.1250,0.0010
45,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,98.1250,0.0015
46,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,98.1250,0.0020
47,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,98.1250,0.0030
48,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,97.8125,0.0100
49,0.001,ThreeLayer,MNIST,True,vanilla,0.1,0.1,linf_pgd,10,88.4375,0.1000
